In [ ]:
%pip install kagglehub

In [ ]:
import shutil
import os
import kagglehub
from layers import SelfAttention
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense
import sys
# add parent directory to Python path for layers.py access
sys.path.append('..')
# splits = {'train': 'data/train-00000-of-00001.parquet',



#           'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}



# train_df = pd.read_parquet(



#     "hf://datasets/google-research-datasets/poem_sentiment/" + splits["train"])



# validation_df = pd.read_parquet(



#     "hf://datasets/google-research-datasets/poem_sentiment/" + splits["validation"])



# test_df = pd.read_parquet(



#     "hf://datasets/google-research-datasets/poem_sentiment/" + splits["test"])



# verse_text, label



# label: 0 = negative, 1 = positive, 2 = no_impact, 3 = mixed



# Télécharger l'archive
path = kagglehub.dataset_download("bittlingmayer/amazonreviews")
print("Chemin de l'archive téléchargée :", path)

# Dossier de destination pour déplacer l'archive
destination_dir = "C:/Users/alexi/aProjets_Code/IA Projet 2/epf-ia-sentiment-analysis/amazonreviews_dataset"
os.makedirs(destination_dir, exist_ok=True)

# Chemin complet de l'archive dans le dossier de destination
# Récupère le nom de l'archive (ex: "archive.zip")
archive_name = os.path.basename(path)
new_archive_path = os.path.join(destination_dir, archive_name)

# Déplacer l'archive vers le dossier de destination
shutil.move(path, new_archive_path)
print(f"Archive déplacée vers : {new_archive_path}")

# Dossier pour extraire l'archive
extract_dir = os.path.join(destination_dir, "amazonreviews")
os.makedirs(extract_dir, exist_ok=True)
print(f"Archive extraite dans : {extract_dir}")

In [147]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
# Training data
X_train = train_df['verse_text'].values
Y_train = train_df['label'].values

# Validation data
X_validation = validation_df['verse_text'].values
Y_validation = validation_df['label'].values

# Test data
X_test = test_df['verse_text'].values
Y_test = test_df['label'].values

# Y_train = to_categorical(Y_train, num_classes=4)
# Y_validation = to_categorical(Y_validation, num_classes=4)
# Y_test = to_categorical(Y_test, num_classes=4)

In [148]:
vocabulary_size = 10000  # choose 10k most-used words for truncated vocabulary
sequence_length = 900

In [149]:
# Tokenize the text
tokenizer = Tokenizer(num_words=vocabulary_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_validation = tokenizer.texts_to_sequences(X_validation)
X_test = tokenizer.texts_to_sequences(X_test)

#One hot encoding
X_train = pad_sequences(sequences=X_train, maxlen=sequence_length,padding='post', truncating='post')
X_validation = pad_sequences(sequences=X_validation, maxlen=sequence_length,padding='post', truncating='post')
X_test = pad_sequences(sequences=X_test, maxlen=sequence_length, padding='post', truncating='post')

In [150]:
def build_Attention_Model(config,vocabulary_size, sequence_length, embedding_dims, batch_size, num_epochs, random_seed, X_train, Y_train, X_validation, Y_validation):
    tf.random.set_seed(random_seed)
    X = Input(shape=(sequence_length,), batch_size=batch_size)

    embedded = Embedding(input_dim=vocabulary_size, output_dim=embedding_dims)(X)

    # Optional Self-Attention Mechanisms
    if config == 1:
        embedded, attention_weights = SelfAttention(size=50,
                                                    num_hops=6,
                                                    use_penalization=False)(embedded)
    elif config == 2:
        embedded, attention_weights = SelfAttention(size=50,
                                                    num_hops=6,
                                                    use_penalization=True,
                                                    penalty_coefficient=0.1)(embedded)

    # Multi-Layer Perceptron
    embedded_flattened = Flatten()(embedded)
    fully_connected = Dense(units=250, activation='relu')(embedded_flattened)

    # Prediction Layer
    Y = Dense(units=1, activation='softmax')(fully_connected)

    # Compile model
    model = Model(inputs=X, outputs=Y)
    model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

#First Model with 0 config

In [151]:
# CONFIG OPTIONS:
# 0: Simple Multi-Layer Perceptron Model
# 1: Simple Multi-Layer Perceptron Model w/ Self-Attention (Non-Penalized)
# 2: Simple Multi-Layer Perceptron Model w/ Self-Attention (Penalized)

In [152]:
random_seed = 42
# choose 1000-word sequences, either pad or truncate sequences to this
embedding_dims = 50      # number of dimensions to represent each word in vector space
batch_size = 100         # feed in the neural network in 100-example training batches
num_epochs = 100
config = 0

In [ ]:
perceptron = build_Attention_Model(config,vocabulary_size, sequence_length, embedding_dims, batch_size, num_epochs, random_seed, train_df['verse_text'], train_df['label'], validation_df['verse_text'], validation_df['label'])

In [ ]:
perceptron.fit(x=X_train, y=Y_train,
          validation_data=(X_validation, Y_validation),
          epochs=num_epochs, batch_size=batch_size)

In [ ]:
y_pred = perceptron.predict(X_validation)

In [ ]:
# import numpy as np

# # Get the predicted class (index of the highest probability)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_test_classes = np.argmax(Y_test, axis=1)

In [ ]:
print(y_pred)
print(Y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create confusion matrix
conf_matrix = confusion_matrix(Y_test, y_pred[:-1])

# Plot confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3'],
            yticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [119]:
def evaluate_model(model, X_test, Y_test):
    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    return loss, accuracy

In [ ]:
print(evaluate_model(perceptron, X_validation, Y_validation))